In [35]:
import fitz
import pandas as pd
from langchain_cohere import CohereEmbeddings

### Preprocessing Pdf

In [36]:
def extract_questions_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    questions = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()

        # Split text into lines
        lines = text.split('\n')

        # Extract lines that are questions (beginning with a bullet point)
        for line in lines:
            if line.strip().startswith('•'):
                questions.append(line.strip()[1:].strip())
    return questions

In [37]:
pdf_path = 'Data/SEC Questionaire 3.pdf'
questions = extract_questions_from_pdf(pdf_path)
questions_df = pd.DataFrame(questions, columns=["Questions"])
questions_df['Questions'].to_list()

['Betreibt das Unternehmen ein Informationssicherheits-Management-System (ISMS)?',
 'Betreibt das Unternehmen ein Cyber Security Management System (CSMS)?',
 'Betreibt das Unternehmen ein Software Update Management System (SUMS)?',
 'Bestehen für diese(s) System(e) eine Zertifizierung, zum Beispiel nach ISO 21434 oder',
 'Existiert im Unternehmen ein benannter Cyber Security Manager?',
 'Gibt es im Unternehmen eine Cyber Security Management Richtlinie?',
 'Wie werden die Mitarbeiter über die Gefahren beim Umgang mit cyber-security',
 'Werden Mitarbeiter geschult und sensibilisiert? (hinsichtlich Cyber Security)',
 'Wurde eine TARA (Threat and Risk Analysis) für die Produkte/Komponenten',
 'Wird vor Auftragsvergabe an Fremdfirmen die Cyber Security der Auftragnehmer',
 'Wie wird die Geheimhaltung bei der Zusammenarbeit mit Fremdfirmen sichergestellt?',
 'Werden interne Audits zur Überprüfung der Cyber Security durchgeführt?',
 'Existiert eine Zutrittssteuerung in Ihrem Unternehmen?',
 '

In [38]:
embeddings_model = CohereEmbeddings(cohere_api_key="yU3vQ4wcV8gbMCvsHngfAa6OfVmnaMQy1YOpVwqF")
embeddings_pdf = embeddings_model.embed_documents(questions_df['Questions'].to_list())
embeddings_pdf[0]

[0.47070312,
 -0.9296875,
 -1.4609375,
 -0.35498047,
 0.62158203,
 -0.66259766,
 -0.8076172,
 1.2910156,
 -1.2099609,
 -1.3232422,
 -0.20129395,
 -1.7587891,
 -0.9741211,
 -1.2128906,
 -0.0619812,
 0.8129883,
 0.23205566,
 -0.10418701,
 0.92822266,
 3.1855469,
 0.09313965,
 -1.6152344,
 -1.7841797,
 -0.63671875,
 1.3271484,
 -0.5102539,
 0.18859863,
 -2.7792969,
 1.6494141,
 0.059295654,
 0.9604492,
 -0.5566406,
 -1.6494141,
 0.13964844,
 1.0585938,
 -1.0585938,
 -0.26367188,
 1.6630859,
 -0.94921875,
 1.5302734,
 0.40673828,
 2.9238281,
 -0.3022461,
 1.3359375,
 -1.0625,
 0.35473633,
 -0.59716797,
 0.91845703,
 1.1767578,
 -1.3046875,
 -0.5620117,
 -1.8720703,
 -0.012496948,
 -0.28735352,
 2.1640625,
 -0.044525146,
 0.18579102,
 -1.8671875,
 1.3261719,
 -1.796875,
 0.18725586,
 -0.03793335,
 -0.33032227,
 -1.2041016,
 -0.24194336,
 -0.34350586,
 1.09375,
 -1.7197266,
 -1.2109375,
 -1.1220703,
 2.0546875,
 0.1451416,
 0.5053711,
 -2.6132812,
 -0.7475586,
 -0.32421875,
 1.1376953,
 -2.7

### Preprocessing Images

In [39]:
import pytesseract
from PIL import Image

# Paths to the images
image_paths = [
    'data/Sec_Questionaire1.png',
    'data/Sec_Questionaire2.png',
    'data/Sec_Questionaire3.png'
]

# Function to extract text from an image
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Extract text from each image
extracted_texts = [extract_text_from_image(image_path) for image_path in image_paths]
extracted_texts

['3.6.1 [Beschreiben Sie Ihre MaSnahmen zur Datensicherung wie Backups, Wiederherstellung von Dateien und | [nn\nFallback-Vorkehrungen.\nAntwort\n\n‘Wie Tange werden die Daten im Produktivsystem vorratig gehalten, und wie lange werden diese Daten\nAnt\n\nAnt\n\n',
 '374\n\nWelche MaGnahmen wurden getroffen, um Ihr Computersystem vor unbefugtem Eindringen zu schatzen\n(z. B. Firewall, Antivirenprogramm, Kennwortschutz)?\n\n“Antwort\nEingesetzt werden 3 Firewalls, ein Antiviren-Programm und ein Kennwortschutz (siehe hierzu X-VA -45-\n2756027 "IT-Nutzung" und X-VA-45-10000012648 "ISMS - Informationstechnik - Sicherheitsmanagement")\n\n372\n\nNach welchen Verfahren werden Zugniffsrechte vergeben?\nWer ist verantwortlich far den Schutz des Computersystems?\n\nAntwort:\n\nBeschrieben im IT-Handbuch des HYDAC Firmenverbundes ist die Benutzerbeantragung. Die Beantragung\nerfolgt ber ein Benutzerformular, das durch den IT-Koorodinator und den Vorgesetzten zwingend\nfreigegeben werden muss. Vera

In [40]:
embeddings_images = embeddings_model.embed_documents(extracted_texts)
embeddings_images

[[2.4746094,
  -1.6162109,
  0.43164062,
  -1.0732422,
  2.8847656,
  -0.41870117,
  -0.045043945,
  -0.46264648,
  1.3076172,
  1.0898438,
  0.82958984,
  -0.37817383,
  -1.3417969,
  -2.328125,
  -1.0976562,
  -1.3017578,
  -0.40722656,
  -0.42651367,
  1.1972656,
  0.1932373,
  -0.60791016,
  0.25805664,
  0.0012111664,
  0.7055664,
  -0.58154297,
  0.8881836,
  -1.5742188,
  -0.9555664,
  1.8701172,
  -0.27856445,
  0.9589844,
  0.16247559,
  -0.1763916,
  1.6005859,
  1.4238281,
  -3.4140625,
  0.08544922,
  1.7480469,
  -0.36889648,
  0.43432617,
  -0.1640625,
  0.25854492,
  -0.5957031,
  -1.0351562,
  -0.95458984,
  0.7949219,
  1.5732422,
  3.2949219,
  0.4753418,
  -1.2597656,
  -0.56347656,
  -0.37036133,
  -0.4489746,
  -0.1239624,
  2.1152344,
  -0.101379395,
  0.05142212,
  -0.6772461,
  -0.25732422,
  -2.0410156,
  -0.22924805,
  -0.296875,
  -1.3007812,
  -0.5366211,
  -0.6977539,
  0.2734375,
  -2.0175781,
  -1.6142578,
  -0.20776367,
  -1.78125,
  1.2890625,
  1.46093

### Preprocessing Excel

In [41]:
file_path = 'Data/Security_Questionnaire_Template.V1.5.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load the sheet that contains the questions and answers
questions_sheet = pd.read_excel(file_path, sheet_name='Questionnaire',header=3)

# Extract the relevant columns for questions and their corresponding answers
questions_answers = questions_sheet[['Questions', "Supplier's Answer"]]

# Drop rows where questions or answers are not present
questions_answers = questions_answers.dropna(subset=['Questions', "Supplier's Answer"])
questions_answers

/Users/omeriqbal/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Questions,Supplier's Answer
0,Does the Supplier/ Service Provider offer cryp...,HYDAC is selling a physical product to .
1,"Is there a physical lockdown (procedure, imple...",All buildings are secured by a fence and locke...
2,Does the Supplier/ Service Provider permit ten...,Only HYDAC IT administrators and management ha...
3,How does the Supplier/ Service Provider ensure...,On all systems are Antivirus scanner with fire...
4,Where are the Supplier/ Service Provider data ...,Germany and the Netherlands
...,...,...
103,Does the Supplier/ Service Provider support gr...,HYDAC administrators are setting the permissio...
104,Does the Supplier / Service Provider provide a...,"HYDAC is not providing a service, application ..."
105,Does the Supplier/ Service Provider service pr...,If an account is used simultaneously from diff...
106,Can the Supplier/ Service Provider confirm tha...,There is a screenlock after 15 minutes. The VP...


In [44]:
excel_questions_embeddings = embeddings_model.embed_documents(questions_answers['Questions'].to_list())
excel_answers_embeddings = embeddings_model.embed_documents(questions_answers["Supplier's Answer"].to_list())
excel_questions_embeddings

[[-0.03704834,
  -0.07080078,
  1.1523438,
  -0.46875,
  -1.0361328,
  0.66845703,
  -1.6835938,
  0.5722656,
  -1.7978516,
  0.019989014,
  0.62939453,
  0.74365234,
  -0.35717773,
  -1.4228516,
  -0.20910645,
  0.24194336,
  0.15307617,
  1.5244141,
  -0.9291992,
  0.35888672,
  0.9448242,
  0.4892578,
  0.5488281,
  -2.515625,
  0.28198242,
  1.9667969,
  -0.67285156,
  -1.1904297,
  -0.21130371,
  0.90527344,
  -0.13366699,
  0.46044922,
  -0.7871094,
  -1.2714844,
  -1.3271484,
  0.4284668,
  0.65722656,
  -0.40966797,
  -1.390625,
  0.08642578,
  1.3349609,
  1.8144531,
  0.2244873,
  0.022506714,
  -2.7519531,
  0.6015625,
  1.8486328,
  0.72509766,
  1.2333984,
  -0.1262207,
  0.76953125,
  -1.5634766,
  -1.7353516,
  -0.52978516,
  0.62646484,
  -0.7133789,
  -4.546875,
  -0.18859863,
  -0.609375,
  -1.0742188,
  1.8222656,
  -0.61865234,
  -0.81884766,
  0.18103027,
  -2.0566406,
  -0.7158203,
  -0.4736328,
  3.0078125,
  0.015823364,
  0.062561035,
  2.0742188,
  0.81640625,